In [ ]:
# IMPORTING DEPENDENCIES

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
# USING MEDIAPIPE TO GENERATE LANDMARKS

In [4]:
# setting mediapipe holistics
#we are creating 2 variables ,one for mediapipe holistic and second one for mediapipe drawing utilities
#mediapipe holistoc and mediapipe drawing utilites are 2 components in the mediapipe 
#mediapipe holistic helps in pose estimation  whereas the mediapipe drawing utilities helps in visualizing the results for the developers
holistics =mp.solutions.holistic #imports the holistic module from the solutions which is a sub module in mediapipe
drawings=mp.solutions.drawing_utils #imports the drawing module from the solutions which is a sub module in mediapipe

In [5]:
def mp_detect(im,model):   #setting a function for detection.here model is mediapipe holistic model
    im=cv2.cvtColor(im,cv2.COLOR_BGR2RGB)  #converts im from BGR 2 RGB color space.cvtcolor() is function that changes imag
    im.flags.writeable=False   #sets im unwriteable so that the mediapipe won't the im data during processing
    results=model.process(im)  #process() is a function . It processes the im using the holistic model
    im.flags.writeable=True    #sets back im to writeable after processing
    im=cv2.cvtColor(im,cv2.COLOR_RGB2BGR)  #converts im from RGB to BGR color space
    return im,results          #return the processed im,detection results


In [6]:
def landmarks_style(im,results):#styling the landmarks like size,color etc
    # Draw face connections and styling face connections
    drawings.draw_landmarks(im, results.face_landmarks, holistics.FACEMESH_TESSELATION,
                             drawings.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                             drawings.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                             ) 
    # Draw pose connections  and styling pose connections
    drawings.draw_landmarks(im, results.pose_landmarks, holistics.POSE_CONNECTIONS,
                              drawings.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4),
                               drawings.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2)
                             ) 
    # Draw left hand connections  and styling left hand connections
    drawings.draw_landmarks(im, results.left_hand_landmarks, holistics.HAND_CONNECTIONS,
                              drawings.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),
                               drawings.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2)
                             ) 
    # Draw right hand connections  and styling right hand connections
    drawings.draw_landmarks(im, results.right_hand_landmarks, holistics.HAND_CONNECTIONS,
                              drawings.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                              drawings.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                             ) 

In [ ]:
cap =cv2.VideoCapture(0)  #accesing webcam.default index of webcam is 0,storing everything got from video in cap avriable
with holistics.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():                   #checking whether the webcam is open or not.so used isOpened() function 
        ret,frame=cap.read()                #reading the feed.while reading, we are getting 2 values they are return value(true/false) ,im i.e frame
        im,results=mp_detect(frame,holistic)
        print(results)
        landmarks_style(im,results)        #draw landmarks
        cv2.imshow('opencv feed',im)     #showing what has read  the user."opencv feed" is name given to frame
        if cv2.waitKey(10) & 0xFF==ord('p'):#breaking the loop.  if the current key pressed is 'q' the loop breaks
            break                                                           
    cap.release()                           #releases the webcam
    cv2.destroyAllWindows()                 #closes all opencv windows

In [ ]:
# KEYPOINT EXTRsign

In [8]:
#np.array is used to create arrays
#pose_landmarks has x,y,z,visibility rest has only x,y,z
#results.pose_landmarks give the result in form of landmark{x:,y:,z:,visibility:} 
#result.pose_lanmarks.landmark give the result in form of list i.e [x: y: z: visibility:, etc]
#using flatten converts into 1-D array

In [9]:
def keypoints_extract(results):   # function to extract key points 
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh]) #cancatenating arrays

In [ ]:
keypoints_extract(results)

In [ ]:
keypoints_extract(results).shape

In [12]:
# CREATING SUBDIRS FOR CLASSES

In [13]:
PATH=os.path.join('ISL_DATASET') #PATH is path where exported data is stored i.e numpy arrays,here the exported data is extracted keypoints
signs=np.array(['hello','thanks','iloveyou','sorry','no','goodbye']) #detecting the signshello ,thanks,iloveyou
no_seq=30                                 #using 30 frames do detect the sign,collecting 30 videos for each sign
seq_len=30                              #each video is of length 30 frames

In [14]:
#ISL_DATASET is directory
#sign detection means takes seq of data to predict the sign rather than single frame

In [15]:
#creating folders to store the data
for sign in signs:  #it iterates over every sign
    for seq in range(no_seq): #it iterates over range of numbers from o to 29(no_seq-1)
        try:
            os.makedirs(os.path.join(PATH,sign,str(seq)))  #it is creating directory for each combination of sign and seq number 
        except:                                                        #using os.makedirs()
            pass                                                        #if directory exists it goes to except block

In [16]:
# COLLECTING PERSONALISED DATA SAMPLES

In [ ]:
cap =cv2.VideoCapture(0) 
with holistics.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    for sign in signs:
        for seq in range(no_seq):
            for frame_num in range(seq_len):
                    ret,frame=cap.read()     #each frame is read from video feed          
                    im,results=mp_detect(frame,holistic) #pose detection is performed on the im using the func and return the landmarks
                    print(results)   #on the im and pose detection results
                    landmarks_style(im,results)   #landmarks are styled on each im    
                    cv2.imshow('opencv feed',im)        #displaying the im
                    if cv2.waitKey(10) & 0xFF==ord('p'):
                        break    
                    if frame_num == 0: 
                             #message displayed started data collection
                             cv2.putText(im, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                             cv2.putText(im, 'Collecting frames for {} Video Number {}'.format(sign, seq), (15,12), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                             # Show to screen
                             cv2.imshow('OpenCV Feed', im)
                             cv2.waitKey(2000)
                    else:
                  
                             #displays the ongoing data collection
                             cv2.putText(im, 'Collecting frames for {} Video Number {}'.format(sign, seq), (15,12), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                             # Show to screen
                             cv2.imshow('OpenCV Feed', im)
                    
                    keypoints = keypoints_extract(results)
                    npy_path = os.path.join(PATH, sign, str(seq), str(frame_num))
                    np.save(npy_path, keypoints) #keypoints extraced from pose detection are saved in numpy arrays and stored in corresponding directory 
                                                #of sign,seq number,frame
    cap.release()        #releases video object                   
    cv2.destroyAllWindows()      #open cv windows are destroyed           

In [ ]:
#collection break: having breaks between each seq collection allows you to reset and reposition yourself to collect the sign from start to finish


In [ ]:
# preprocess data create labels and features

In [17]:
from tensorflow.keras.utils import to_categorical # to_categorical : conerts integer into categorical data
from sklearn.model_selection import train_test_split # train_test_split : splits the dataset into training and testing

In [18]:
#creating a label array that represents signs
labels_map={label:num for num,label in enumerate(signs)}

In [ ]:
labels_map

In [20]:
seqs,labels=[] ,[] #seqs represents x-data ,labels represents y-data
for sign in signs:
    for seq in range(no_seq):
        window=[]
        for frame_num in range(seq_len):
            res=np.load(os.path.join(PATH,sign,str(seq),"{}.npy".format(frame_num)))
            window.append(res)
        seqs.append(window)
        labels.append(labels_map[sign])

In [ ]:
seqs

In [ ]:
#preprocesing
X=np.array(seqs)
X

In [ ]:
labels

In [ ]:
y=to_categorical(labels).astype(int) #binary format
y

In [50]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.05)

In [ ]:
X_train.shape,X_test.shape ,y_train.shape,y_test.shape#training and testing tuples size

In [52]:
# MODEL TRAINING

In [53]:
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
#LSTM is a rnn layer which helps in handling the sequential data
#Dense is a fully connected neural network layer.each neuron is connected to each neuron in previous layer ,helps in learning the complex patterns
#TensoBoard is a visualization tool for monitoring and visualizing various aspects like model training and evaluation
#sequential helps to build neural networks layer by layer in a sequential manner.layers are added to model one by one 
#and each is connected to previous layers

In [54]:
logS_dir = os.path.join('Logs')#creates the directory path named logs
tb_callback = TensorBoard(log_dir=logS_dir) # tb_callback is a tensorboard callback object.tensorboard writes the log files at path logs_dir

In [55]:
#Training the model
#A sequential model architecture consists of multiple LSTM layers followed by dense layers.the activation function 'relu' is
#used in LSTM and dense layers except last dense layer that uses softmax activation function.softmax is used for multi class classification
#the model is designed to take input seq 30 with 1662 features
#return_seq =false means final output should be returned ,true means it should return seq of outputs not only final output 
#64,128,32 means neurons in the layer
#activation functions decide whether the neuron should be activated or not.they are applied on output of each neuron
#Relu =max(0,x)
#softmax used in output layer in multi classification models to produce probabilites of each class

In [58]:
model=Sequential() #sequential model
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662))) 
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(signs.shape[0],activation='softmax'))

In [59]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy']) #model compiling
#optimizer:updates the weigths of neural network during training,Adam is an algorithm
#loss function:it finds the difference between actual and predicted output.categorical_crossentropy is a loss function used for multiclassclassification
#metrics:used to evaluate the performance of model.metric used is categorical_accuracy.it is used in multi class classification

In [ ]:
history=model.fit(X_train,y_train,epochs=200,validation_split=0.2,callbacks=[tb_callback])
#epochs:The number of times the entire training dataset will be passed forward and backward through the neural network during training

In [61]:
res = [.7, 0.1, 0.2,.3,.5,.4]

In [ ]:
signs[np.argmax(res)]

In [ ]:
signs.shape[0] #output layer has 3 neurons

In [ ]:
model.summary()

In [65]:
#prediction

In [ ]:
res=model.predict(X_test)
print(res)

In [ ]:
signs[np.argmax(res[0])]

In [68]:
#saving weights

In [69]:
model.save('sign.keras')

In [70]:
#metrics

In [71]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)
print(yhat)

In [73]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)


In [76]:
# step 11 test in real time

In [77]:
color = [(245,117,16), (117,245,16), (16,117,245),(16,119,245),(16,117,243),(17,117,245)]
def prob_viz(res, signs, input_frame, color):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), color[num], -1)
        cv2.putText(output_frame, signs[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
# 1. New detection variables
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
seq = []
sentence = []
threshold = 0.4

cap = cv2.VideoCapture(0)

with holistics.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        im, results = mp_detect(frame, holistic)
        print(results)
        
     
        landmarks_style(im, results)
        
        
        keypoints = keypoints_extract(results)
           #seq.insert(0,keypoints)
          #seq = seq[:30]
        seq.append(keypoints)
        seq = seq[-30:]
        
        if len(seq) == 30:
            res = model.predict(np.expand_dims(seq, axis=0))[0]
            print(signs[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if signs[np.argmax(res)] != sentence[-1]:
                        sentence.append(signs[np.argmax(res)])
                else:
                    sentence.append(signs[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

         
            im = prob_viz(res, signs, im, color)
            
        cv2.rectangle(im, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(im, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', im)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('p'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
from sklearn.metrics import precision_score, recall_score
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

precision = precision_score(y_test_labels, y_pred_labels, average='weighted')
recall = recall_score(y_test_labels, y_pred_labels, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test_labels, y_pred_labels, labels=range(len(signs)), target_names=signs)
print(report)


In [ ]:
# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')